# Busquedas semanticas y RAG

Con la base de datos vectorial podemos brindar al modelo información adicional para que responda según el mejor contexto posible, esto nos ayuda a que el modelo responda con datos con los que no fue entrando y que la ventana de contexto no sea tan grande y sin información innecesaria.

In [1]:
import os
import openai
import chromadb
from dotenv import load_dotenv

In [2]:
# Cargar variables de entorno
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Instanciamos el cliente para acceder al modelo generador de texto LLM

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [4]:

# Configuración
EMBEDDINGS_DIR = "../data/03_embeddings/"
DB_NAME = "chroma_db"
EMBEDDING_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-4"

In [5]:
# Inicializar ChromaDB
chroma_client = chromadb.PersistentClient(path=EMBEDDINGS_DIR)
collection = chroma_client.get_or_create_collection(DB_NAME)

In [6]:
# Función para obtener embeddings
def get_embedding(text):
    """Obtiene el embedding de un texto usando OpenAI."""
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002" #text-embedding-ada-002
    )
    return response.data[0].embedding
    

In [8]:
# Función para realizar la busqueda semantica 
# Esta metodologia compara el embedding generado con el texto de entrada (prompt) con los embeddings que estan en la base
# Buscando los más similares entre si mediante la similitud del coseno
def search_documents_with_model(query, k=15):
    """Realiza una búsqueda en la base de datos vectorial utilizando un modelo específico para la consulta."""
    query_embedding = get_embedding(query)
    results = collection.query(query_embeddings=[query_embedding], n_results=k)
    
    return results

In [9]:
# Función para generar respuestas con RAG
# Ya con el texto de los chunks con información más relevantes podemos usarlos como contexto para que el modelo
# Responda de manera correcta
def generate_rag_response(query, k=15):
    """Recupera documentos relevantes y genera una respuesta con el LLM."""
    relevant_docs = search_documents_with_model(query, k)
    context = "\n".join([doc for doc in relevant_docs['documents'][0]])
    prompt = f"""
    Usa la siguiente información para responder la pregunta:
    
    {context}
    
    Pregunta: {query}
    """
    
    response = client.chat.completions.create(
        model=LLM_MODEL,
        messages=[{"role": "system", "content": "Eres un asistente virtual de la compañia de financiamiento Tuya S.A"},
                  {"role": "user", "content": prompt}],
        temperature=0.1
    )
    return response.choices[0].message.content

In [14]:
# Prueba de búsqueda y generación
# Pregunta 1 requerida en la prueba tecnica
def test_rag():
    query = "¿ Cuáles son los nombres de las tarjetas que tiene disponibles Tuya S.A?."
    response = generate_rag_response(query)
    print("Respuesta generada:")
    print(response)

# Ejecutar prueba
test_rag()


Respuesta generada:
Las tarjetas de crédito disponibles de Tuya S.A son:

1. Tarjeta Éxito
2. Tarjeta Carulla
3. Tarjeta Éxito Pro Mastercard
4. Tarjeta Éxito Gold Mastercard
5. Tarjeta Carulla Gold Mastercard
6. Tarjeta Carulla Mastercard Black
7. Tarjeta Club del Comerciante Mastercard
8. Tarjeta Éxito Digital Mastercard

Por favor, ten en cuenta que la disponibilidad de estas tarjetas puede variar y es recomendable verificar la información actualizada en el sitio web oficial de Tuya S.A.


In [15]:
# Prueba de búsqueda y generación
# Pregunta 2 requerida en la prueba tecnica
def test_rag():
    query = "¿ Cuáles son los valores la tasa de interés y póliza del producto credicompras?."
    response = generate_rag_response(query)
    print("Respuesta generada:")
    print(response)

# Ejecutar prueba
test_rag()

Respuesta generada:
El producto Credicompras tiene diferentes tasas de interés y pólizas dependiendo del tipo de crédito. Para el Credicompras Dirigido y Credicompras Libre Destino, la Tasa Efectiva Anual es de 24,89805% y la póliza colectiva de seguro de deudores es de 3.143. Para el Credicompras Bajo Monto, la Tasa Efectiva Anual es de 56,08602% y la póliza colectiva de seguro de deudores también es de 3.143.
